In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai_deployment_id = os.getenv("OPENAI_DEPLOYMENT_ID")

In [2]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""

prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""

print(prompt)


Summarize the text delimited by triple backticks \ 
into a single sentence.
```
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
```



In [6]:
messages = [{"role": "user", "content": prompt}]
response = openai.ChatCompletion.create(
    deployment_id=openai_deployment_id,
    messages=messages,
    temperature=0, # this is the degree of randomness of the model's output
)

print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Clear and specific instructions should be provided to guide a model towards the desired output, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs.",
        "role": "assistant"
      }
    }
  ],
  "created": 1686927236,
  "id": "chatcmpl-7S57k5BJHjkIqzR9NsBgvFJfoCkgy",
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 37,
    "prompt_tokens": 135,
    "total_tokens": 172
  }
}


In [7]:
response.keys()

dict_keys(['id', 'object', 'created', 'model', 'choices', 'usage'])

In [10]:
print(response['choices'])

[<OpenAIObject at 0x10edb2e70> JSON: {
  "finish_reason": "stop",
  "index": 0,
  "message": {
    "content": "Clear and specific instructions should be provided to guide a model towards the desired output, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs.",
    "role": "assistant"
  }
}]


In [11]:
response.choices[0].message["content"]

'Clear and specific instructions should be provided to guide a model towards the desired output, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs.'

In [12]:
response['usage'], response['usage']['total_tokens']

(<OpenAIObject at 0x10edb3b30> JSON: {
   "completion_tokens": 37,
   "prompt_tokens": 135,
   "total_tokens": 172
 },
 172)

## Helper Function

In [8]:
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        deployment_id=openai_deployment_id,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [9]:
response = get_completion(prompt)
print(response)

To guide a model towards the desired output and reduce the chances of irrelevant or incorrect responses, it is important to provide clear and specific instructions, which may require longer prompts for more clarity and context.


In [ ]:
    messages=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "Who were the founders of Microsoft?"}
    ]

In [22]:
prompt = f"""what is the answer to life, the universe, and everything?"""

messages=[
    {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
    {"role": "user", "content": prompt}
]

response = openai.ChatCompletion.create(
  deployment_id=openai_deployment_id,
  messages = messages,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)

response

<OpenAIObject chat.completion id=chatcmpl-7S4rotYm1PPU9kkGv26UVV5DcM2kn at 0x113ff1d30> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "According to Douglas Adams' book \"The Hitchhiker's Guide to the Galaxy,\" the answer to the ultimate question of life, the universe, and everything is \"42.\" However, the meaning behind this answer is never explained in the book.",
        "role": "assistant"
      }
    }
  ],
  "created": 1686926248,
  "id": "chatcmpl-7S4rotYm1PPU9kkGv26UVV5DcM2kn",
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 49,
    "prompt_tokens": 37,
    "total_tokens": 86
  }
}

In [29]:
prompt = f"""what is the answer to life, the universe, and everything?"""

def get_completion(prompt):
    messages=[
            {"role": "system", "content": "You are an AI assistant that helps people find information."},
            {"role": "user", "content": prompt}
        ]

    response = openai.ChatCompletion.create(
        deployment_id=openai_deployment_id,
        messages = messages,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)

    return response.choices[0].message["content"]

response = get_completion(prompt)
print(response)

According to Douglas Adams' book "The Hitchhiker's Guide to the Galaxy", the answer to life, the universe, and everything is 42. However, this is a fictional answer and its significance is never fully explained in the book. In reality, the meaning of life and the universe is a philosophical question that has been debated for centuries with no definitive answer.


Top P: 

Similar to temperature, this controls randomness but uses a different method. Lowering Top P will narrow the model’s token selection to likelier tokens. Increasing Top P will let the model choose from tokens with both high and low likelihood. Try adjusting temperature or Top P but not both.